# notebook to interpolated NEMO output on ISMIP6 grid

## load module

In [1]:
import numpy as np
import os
import copy
from datetime import datetime

# eos
import gsw

# data
import netCDF4 as nc                             # read netcdf
import xarray as xr                              # write netcdf
import pyproj                                    # projection

# plot
import cartopy
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt 

# interpolation
from scipy.interpolate import griddata
from scipy.interpolate import LinearNDInterpolator

## functions

### function to test file presence

In [2]:
def isfile(cfl):
    """
    Purpose: test if all the file in the input file list exist
    
    Args:
        cfl: list of input file name (string)
        
    Return: None
    
    Raise:
        FileError: file is missing
    """

    nerr=0
    cferr=''
    for cfile in cfl:
        if not os.path.isfile(cfile):
            print ("File {} does not exist".format(cfile))
            cferr=cferr+' '+cfile
            nerr=nerr+1
    
    if nerr > 0:
        raise RuntimeError('At least one file is missing '+cferr)

### function to interpolate data and mask

In [3]:
def interpolate_data(xsrc,ysrc,datasrc,masksrc,xtrg,ytrg):
    """
    Purpose: interpolate datasrc and masksrc on trg grid
    
    Args:
        xsrc   : X coordinates of the source grid [flatten array]
        ysrc   : Y coordinates of the source grid [flatten array]
        datasrc: data to interpolate on the source grid [flatten array]
        
        xtrg : X coordinates of the trg grid [2d array]
        ytrg : Y coordinates of the trg grid [2d array]
        
    Return: 
        datatrg : datasrc interpolated on the trg grid
        masktrg : masksrc interpolated on the trg grid
    """
    
    print('    interpolate data and mask ...')
    
    # interpolation of  data
    datatrg = griddata((xsrc,ysrc), datasrc, (xtrg, ytrg), method='linear')
    
    # interpolation of mask
    masktrg = griddata((xsrc,ysrc), masksrc, (xtrg, ytrg), method='nearest')
    
    # mask interpolated data
    datatrg[masktrg<1.0]=np.nan
    
    return datatrg, masktrg

### function to extrapolate by 1 cell

In [4]:
def extrapolation_data(data):
    """
    Purpose: extrapolate by one cell the data (use the nan mean over the 8 boundary cells)
    
    Args:
        data: data to interpolate on the source grid [2d array]
        
    Return: 
        data_ext : data extrapolated by 1 cell [2d array]
    """
    
    print('    extrapolate by 1 cell ...')
    
    data_ext=copy.deepcopy(data)
    
    # find all masked data
    idx=np.where(np.isnan(data))
    
    # fill masked data if they have at least one valid value around
    ndata=len(idx[0])
    for iidx in range(ndata):
        ii=idx[0][iidx]
        jj=idx[1][iidx]
        data_ext[ii,jj]=np.nanmean(data[ii-1:ii+2,jj-1:jj+2])
        
    return data_ext

### function to convert CT and SA to pt0 et practical salinity

In [5]:
def convert_from_TEOS10_to_EOS80(ct,sa,z,lon,lat):
    """
    Purpose: convert conservative temperature and absolute salinity to potential temperature and practical salinity
    
    Args:
        ct  : conservative temperature array [2D or 3D array]
        sa  : absolute salinity array [same dimension as ct]
        z   : depth array [same dimention as ct]
        lon : longitude array
        lat : latitude array
         
    Return: 
        sp  : practical salinity unit
        pt0 : potential temperature reference to surface
    """
    sp =gsw.conversions.SP_from_SA(sa,z,lon,lat)
    pt0=gsw.conversions.pt_from_CT(sa,ct)
    return sp,pt0

In [6]:
def vertical_interpolation(data3din,zin,zout, jkout):
    """
    Purpose: compute conservative vertical interpolation (bottom and top value use persistence)
    
    Args:
        data3din : 3d array on src vertical discretisation
        zin      : 1d array of src vertical level
        zout     : 1d array of trg vertical level
        jkout    : vertical level to proceed on output grid
         
    Return: 
        data2d   : data interpolated on jk output level
    """  
    
    print('    vertical interpolation ...')
    
    nkin, njin, niin=data3din.shape
    
    data3din_ext = copy.deepcopy(data3din)
    ztmp=np.zeros(shape=(nkin,))
    zweight=np.zeros(shape=(nkin,njin,niin))
    zdata=np.zeros(shape=(njin,niin))

    # find all w level in a ISMIP6 level
    jkmin=np.where(zin <= zout[jkout])[0].tolist()[-1]
    jkmax=np.where(zin >= zout[jkout+1])[0].tolist()[0] - 1
    
    # get list level for up and bottom
    jkin=[*range(jkmin,jkmax+1)]
    jkinp1=[*range(jkmin+1,jkmax+2)]

    # extrapolation top data (by 1 cell) : persistence method
    for jk in reversed(jkin):
        zref = data3din[jk+1,:,:]
        zdata[:,:] = data3din[jk,:,:]
        zdata[np.isnan(zdata)] = zref[np.isnan(zdata)]
        data3din_ext[jk,:,:] = zdata[:,:]

    # extrapolation bottom data (by 1 cell) : persistence method
    for jk in jkin:
        zref = data3din_ext[jk-1,:,:]
        zdata[:,:] = data3din_ext[jk,:,:]
        zdata[np.isnan(zdata)] = zref[np.isnan(zdata)]
        data3din_ext[jk,:,:] = zdata[:,:]

    # mask data3din_ext
    data3din_ext=np.ma.masked_invalid(data3din_ext)

    # define depth of top and bottom boundary (take care of partially included)
    zup = zin[jkin]  ; zup[0]   = zout[jkout]
    zbot= zin[jkinp1]; zbot[-1] = zout[jkout+1]

    # now we can compute weight
    ztmp[jkin] = (zbot - zup) / (zout[jkout+1] - zout[jkout])
    zweight[:,:,:]=ztmp[:,None,None]    

    # compute mean value
    zdata = np.ma.average(data3din_ext,weights=zweight,axis=0)

    # mask data out
    mask2dout = np.ones(shape=(njin,niin))
    zweight[np.isnan(data3din)]=0.
    mask2dout[np.sum(zweight,axis=0)<0.5] = np.nan         # mask 0.0 (some variable in the netcdf are not masked properly)
    
    # the end
    return zdata.data[:,:] * mask2dout[:,:]

In [7]:
def process_data(tplin):
    """
    Purpose: process all the data (vertical interpolation if needed, then horizontal interpolation)
    
    Args:
        tplin : tuple containing the variable dictionary [0], and the grid tuple data [1]
        grid tuple : tuple containing (xin[1d],yin[1d],zin[1d],xout[2d],yout[2d],zout[1d],sy[slice on y (input)],sz[slice on z (input)])
         
    Return: 
        da    : data interpolated on output grid in a xr.dataarray
    """
    
    # set input data (need to do this this way to use dask bag map)
    # variable dictionary
    dvar=tplin[0]
    
    # grid data
    xin,yin,zin,x2dout,y2dout,zout,sy,sz = tplin[1]
    
    print('process '+dvar['NEMOvar']+' ...')

    print('    read data ...')
    
    cf_data_in=dvar['NEMOfile']
    isfile([cf_data_in])
    ncid= nc.Dataset(cf_data_in,'r')

    # define variables name
    cvarin = dvar['NEMOvar']
    cvarout= dvar['ISMIP6var']
    
    # get n dimension
    shape = dvar['shape']
    
    # read data
    if shape == '2D':
        data2din = ncid.variables[cvarin][0,sy,:].squeeze().filled(np.nan)
    elif shape == '3D':
        data3din = ncid.variables[cvarin][0,sz,sy,:].squeeze().filled(np.nan)
    else:
        print('ERROR: shape unknown')
        raise
        
    # close file
    ncid.close()
    
    if shape == '2D':
        # define mask
        mask2din = np.ones(shape=data2din.shape)
        mask2din[data2din==0.0] = 0         # mask 0.0 (some variable in the netcdf are not masked properly)
        mask2din[np.isnan(data2din)] = 0    # mask nan (ie missing value data)
        maskin = mask2din.flatten()

        # extrapolate data
        datain = extrapolation_data(data2din).flatten()

        # interpolate data and mask
        dataout, maskout = interpolate_data(xin,yin,datain.flatten(),maskin.flatten(),x2dout,y2dout)
        
    elif shape == '3D':

        data2din=np.zeros(shape=data3din[0,:,:].shape)
        dataout=np.zeros(shape=(zout.shape[0]-1,x2dout.shape[0],x2dout.shape[1]))
        maskout=np.zeros(shape=(zout.shape[0]-1,x2dout.shape[0],x2dout.shape[1]))
        for jk in range(len(zout)-1):
            
            print(jk,'/',len(zout)-2)
            
            # vertical interpolation
            data2din=vertical_interpolation(data3din,zin,zout,jk)
            
            # define mask
            mask2din = np.ones(shape=data2din.shape)
            mask2din[data2din==0.0] = 0         # mask 0.0 (some variable in the netcdf are not masked properly)
            mask2din[np.isnan(data2din)] = 0    # mask nan (ie missing value data)
            maskin = mask2din.flatten()

            # extrapolate data
            datain = extrapolation_data(data2din).flatten()

            # interpolate data and mask
            dataout[jk,:,:], maskout[jk,:,:] = interpolate_data(xin,yin,datain.flatten(),maskin.flatten(),x2dout,y2dout)

    # define new attributes
    dvar['att']['valid_min'] = np.nanmin(dataout)
    dvar['att']['valid_max'] = np.nanmax(dataout)

    # define dataarray
    da = xr.DataArray(
            data   = np.ma.masked_invalid(dataout),
            dims   = dvar['dims'],
            attrs  = dvar['att'],
        )

    return (da, cvarout)

## Main

### define input data

In [8]:
# do you want to use dask (3 cores it works well on my Mac)
luse_dask=True

# NEMO coordinates file
cf_coord_NEMO='eORCA025.L121-OPM021_mesh_mask.nc'

# ISMIP6 coordinates file
cf_coord_ISMIP6='ISMIP6_ocean_grid.nc'

# projection definition
cprojNEMO='epsg:4326'
cprojISMIP6='epsg:3031'

ctag='y2029.10y'
cconfcase='eORCA025.L121-OPM021'

# output file
cf_out='ISMIP6_{}_{}_WP1_TiPACCs.nc'.format(cconfcase,ctag)

# global attributes
dgatt = {'source' :'{} NEMO simulation'.format(cconfcase),
         'model time' : '10 years average starting 01/01/2029',
         'grid'   : 'ISMIP6',
         'contact':'P. Mathiot (IGE)',
         'creation date':'{}'.format(datetime.now()),
        }

# dict. of data for each variable
dthetao={
    'NEMOvar'   : 'votemper',
    'NEMOfile'  : '{}_{}_gridT.nc'.format(cconfcase,ctag),
    'ISMIP6var' : 'thetao',
    'shape'     : '3D',
    'dims'      : ['z','y','x'],
    'att' : dict(
          long_name="Sea water potential temperature",
          units="°C",
          _FillValue=-9999.99,
          grid_mapping='crs',
     ), 
     }

dso={
    'NEMOvar'   : 'vosaline',
    'NEMOfile'  : '{}_{}_gridT.nc'.format(cconfcase,ctag),
    'ISMIP6var' : 'so',
    'shape'     : '3D',
    'dims'      : ['z','y','x'],
    'att' : dict(
          long_name="Sea water salinity",
          units="g/kg",
          _FillValue=-9999.99,
          grid_mapping='crs',
     ), 
     }

dsbt={
    'NEMOvar'   : 'sosbt',
    'NEMOfile'  : '{}_{}_gridT.nc'.format(cconfcase,ctag),
    'ISMIP6var' : 'tob',
    'shape'     : '2D',
    'dims'      : ['y','x'],
    'att' : dict(
          long_name="Sea water potential temperature",
          units="°C",
          _FillValue=-9999.99,
          grid_mapping='crs',
     ), 
     }

dsbs={
    'NEMOvar'   : 'sosbs',
    'NEMOfile'  : '{}_{}_gridT.nc'.format(cconfcase,ctag),
    'ISMIP6var' : 'sob',
    'shape'     : '2D',
    'dims'      : ['y','x'],
    'att' : dict(
          long_name="Sea water salinity at sea floor",
          units="g/kg",
          _FillValue=-9999.99,
          grid_mapping='crs',
     ), 
     }

dfwf={
    'NEMOvar'   : 'sowflisf_cav',
    'NEMOfile'  : '{}_{}_flxT.nc'.format(cconfcase,ctag),
    'ISMIP6var' : 'ficeshelf',
    'shape'     : '2D',
    'dims'      : ['y','x'],
    'att' : dict(
          long_name="ice-shelf melt rates",
          units="kg/m2/s",
          _FillValue=-9999.99,
          grid_mapping='crs',
     ),
     }

disfd={
    'NEMOvar'   : 'isfdraft',
    'NEMOfile'  : '{}_mesh_mask.nc'.format(cconfcase),
    'ISMIP6var' : 'depfli',
    'shape'     : '2D',
    'dims'      : ['y','x'],
    'att' : dict(
          long_name="Depth of floating ice base below geoid",
          units="m",
          _FillValue=-9999.99,
          grid_mapping='crs',
     ), 
     }

dbathy={
    'NEMOvar'   : 'bathy_metry',
    'NEMOfile'  : '{}_mesh_mask.nc'.format(cconfcase),
    'ISMIP6var' : 'deptho',
    'shape'     : '2D',
    'dims'      : ['y','x'],
    'att' : dict(
          long_name="Sea floor depth below geoid",
          units="m",
          _FillValue=-9999.99,
          grid_mapping='crs',
     ), 
     }


dprj={
    'att' : dict(
          grid_mapping_name='polar_stereographic',
          latitude_of_projection_origin=-90.0,
          standard_parallel=-71.0,
          false_easting=0.0,
          false_northing=0.0,
          epsg_code='epsg:3031',
            )
    }
    
ddat_lst=[dthetao,dso,dsbt,dsbs,dfwf,disfd,dbathy]

### load ISMIP6 grid

In [9]:
print('load ISMIP6 coordinates ...')

isfile([cf_coord_ISMIP6])
ncid= nc.Dataset(cf_coord_ISMIP6,'r')

xISMIP6 = ncid.variables['x'][:].squeeze()
yISMIP6 = ncid.variables['y'][:].squeeze()
x2dISMIP6, y2dISMIP6 = np.meshgrid(xISMIP6,yISMIP6)

zbISMIP6 = ncid.variables['z_bnds'][:].squeeze()
zISMIP6 = -np.append(zbISMIP6[:,0],zbISMIP6[:,-1][-1])

lonISMIP6 = ncid.variables['longitude'][:,:].squeeze()
latISMIP6 = ncid.variables['latitude'][:,:].squeeze()

# ISMIP6 is a south stereographic grid =. there is a lat max
latmaxISMIP6 = latISMIP6.max()

ncid.close()

load ISMIP6 coordinates ...


/usr/local/anaconda3/envs/NEMO_to_ISMIP6/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/anaconda3/envs/NEMO_to_ISMIP6/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys
/usr/local/anaconda3/envs/NEMO_to_ISMIP6/lib/python3.7/sit

### create output file structure

In [10]:
# start creating output dataset
ds=xr.open_dataset(cf_coord_ISMIP6).set_coords(['longitude','latitude'])
xcoord=ds['x']
xcoord.attrs = dict(
            long_name="x coordinate of projection",
            standard_name="projection_x_coordinate",
            axis='X',
            )

ycoord=ds['y']
ycoord.attrs = dict(
            long_name="y coordinate of projection",
            standard_name="projection_y_coordinate",
            axis='Y',
            )

print('')

### load NEMO coordinates

In [11]:
print('load NEMO coordinate ...')

isfile([cf_coord_NEMO])
ncid= nc.Dataset(cf_coord_NEMO,'r')
rlatNEMO = ncid.variables['gphit'][:,:].squeeze()

# get max y to avoid processing data too far away from ISMIP6 area
imax=np.max(np.where(rlatNEMO < latmaxISMIP6)[0])
syNEMO=slice(0,imax+1)
print('    only load data for j between 0 and '+str(imax))

# extract data
rlatNEMO = ncid.variables['gphit'][0,syNEMO,:].squeeze()
rlonNEMO = ncid.variables['glamt'][0,syNEMO,:].squeeze()

# extract bathy and isf draft to compute ISMIP6 mbathy and misfd
rbathyNEMO = ncid.variables['bathy_metry'][0,syNEMO,:].squeeze()
risfdNEMO = ncid.variables['isfdraft'][0,syNEMO,:].squeeze()
mbathyNEMO = ncid.variables['mbathy'][0,syNEMO,:].squeeze()
misfdNEMO = ncid.variables['misf'][0,syNEMO,:].squeeze()

# get max z to avoid processing data too far away from ISMIP6 area
rdepwNEMO = ncid.variables['gdepw_1d'][0,:].squeeze()
kmax=np.min(np.where(rdepwNEMO > zISMIP6[-1]))
szNEMO=slice(0,kmax+1)

# extract gdepw
print('    only load data for k between 0 and '+str(kmax))
rdepwNEMO = ncid.variables['gdepw_1d'][0,szNEMO].squeeze()
rdeptNEMO = ncid.variables['gdept_1d'][0,szNEMO].squeeze()

# close file
ncid.close()
print('')

load NEMO coordinate ...
    only load data for j between 0 and 440


/usr/local/anaconda3/envs/NEMO_to_ISMIP6/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """
/usr/local/anaconda3/envs/NEMO_to_ISMIP6/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  del sys.path[0]
/usr/local/anaconda3/envs/NEMO_to_ISMIP6/lib/pyth

    only load data for k between 0 and 96



/usr/local/anaconda3/envs/NEMO_to_ISMIP6/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/anaconda3/envs/NEMO_to_ISMIP6/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/anaconda3/envs/NEMO_to_ISMIP6/lib/python3.7/site-packages/ipy

### convert NEMO coordinates to stereographic coordinates

In [12]:
print('convert NEMO {} coordinates to {} coordinates ...'.format(cprojNEMO,cprojISMIP6))
transformer_lltoxy=pyproj.Transformer.from_crs(cprojNEMO,cprojISMIP6,always_xy=True)
x2dNEMO,y2dNEMO = transformer_lltoxy.transform(rlonNEMO,rlatNEMO)
xNEMO=x2dNEMO.flatten()
yNEMO=y2dNEMO.flatten()
print('')

convert NEMO epsg:4326 coordinates to epsg:3031 coordinates ...



## Interpolate

In [13]:
# grid definition (input and output) gather in the same object
grid_data=(xNEMO,yNEMO,rdepwNEMO,x2dISMIP6,y2dISMIP6,zISMIP6,syNEMO,szNEMO)

### define list of work to do

In [14]:
# build process_data input tuple
dat_lst=[]
for dvar in ddat_lst:
    dat_lst.append((dvar,grid_data))

### start processing data

In [15]:
# if dask is used start a local cluster
print(datetime.now())
client='no_dask'
if luse_dask :
    
    import dask.bag as db  
    from dask.distributed import Client, LocalCluster

    cluster = LocalCluster(
        n_workers=3,
        )

    client=Client(cluster)

2021-07-08 12:14:28.536209


In [16]:
client

Client Scheduler: tcp://127.0.0.1:50388 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 9 Memory: 16.00 GiB


In [17]:
# do the work (dask or non-dask processing)
if luse_dask:
    # bag definition
    dbag=db.from_sequence(dat_lst, npartitions=len(dat_lst))
    dbag_final=dbag.map(process_data)

    # process data
    tout_lst = dbag_final.compute()
else:
    tout_lst=[]
    for tdat in dat_lst:

        # process data
        tout = process_data(tdat) 

        # add data to dataset
        tout_lst.append(tout)

print(datetime.now())
print('')

2021-07-08 12:27:33.659368



### convert to teos10

In [18]:
# decide if we need to convert data (presence of specific variable name in ddat_lst)
lbot=False; loce=False

if (dsbt in ddat_lst) and (dsbs in ddat_lst) and (dbathy in ddat_lst):
    lbot=True

if (dthetao in ddat_lst) and (dso in ddat_lst):
    loce=True
    
# retreive data
for ddat in tout_lst[:]:
    if ddat[1] == 'tob' :
        ctob=ddat[0].values
    if ddat[1] == 'sob' :
        saob=ddat[0].values
    if ddat[1] == 'thetao':
        cto = ddat[0].values
    if ddat[1] == 'so':
        sao = ddat[0].values
    if ddat[1] == 'deptho':
        zbot = ddat[0].values

# convert with teos 10
if lbot:
    print('    convert bottom data from eos10 to eos80')
    spob,tp0ob=convert_from_TEOS10_to_EOS80(ctob,saob,zbot,lonISMIP6,latISMIP6)
    
    for ddat in tout_lst[:]:
        if ddat[1] == 'tob' :
            ddat[0].values = tp0ob
        if ddat[1] == 'sob' :
            ddat[0].values = spob
    
if loce:
    print('    convert ocean data from eos10 to eos80')
    z2d=np.ones(shape=lonISMIP6.shape)
    for jk in range(len(zISMIP6[0:-1])):
        z2d = zISMIP6[jk]
        sao[jk,:,:],cto[jk,:,:]=convert_from_TEOS10_to_EOS80(cto[jk,:,:],sao[jk,:,:],z2d,lonISMIP6,latISMIP6)
    
    for ddat in tout_lst[:]:
        if ddat[1] == 'thetao':
            ddat[0].values = cto
        if ddat[1] == 'so':
            ddat[0].values = sao

    convert bottom data from eos10 to eos80
    convert ocean data from eos10 to eos80


### set missing value in data set and any other modifications (convention, units ...)

In [19]:
# set right convention for some variables and compute min_val and max val
for ddat in tout_lst[:]:
    
    # temporary array
    zdat = np.ma.masked_invalid(ddat[0].values)
    
    # change convention if needed
    if ddat[1] == 'deptho':
        print('    change topography sign')
        zdat = - zdat
    if ddat[1] == 'depfli':
        print('    change ice shelf draft sign')
        zdat = - zdat
    if ddat[1] == 'ficeshelf':
        print('    change ice shelf melt sign')
        zdat = - zdat
    
    # compute min max
    ddat[0].attrs['valid_min']=np.min(zdat)
    ddat[0].attrs['valid_max']=np.max(zdat)
    
    # fill data
    ddat[0].values = zdat.filled(fill_value=-9999.99)

    change ice shelf melt sign
    change ice shelf draft sign
    change topography sign


### write netcdf

In [20]:
# write output
print('write data ...')
for tout in tout_lst:
    cvarISMIP6=tout[1]
    print('    add '+cvarISMIP6+' ...')
    ds[cvarISMIP6]=tout[0]

write data ...
    add thetao ...
    add so ...
    add tob ...
    add sob ...
    add ficeshelf ...
    add depfli ...
    add deptho ...


In [21]:
# write coordinates and attributes
print('    write coordinates')
ds['x']=xcoord
ds['y']=ycoord

print('    write coordinates')
ds['crs'] = 'epsg:3031'
ds['crs'].attrs = dprj['att']

print('    write global att')
ds.attrs = dgatt

print('    write netcdf ...')
ds.to_netcdf(cf_out)
print('--------------------------------')

    write coordinates
    write coordinates
    write global att
    write netcdf ...
--------------------------------


In [22]:
# look at structure
ds

<xarray.Dataset>
Dimensions:    (nbounds: 2, x: 761, y: 761, z: 30)
Coordinates:
  * x          (x) float64 -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
  * y          (y) float64 -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
  * z          (z) float64 -30.0 -90.0 -150.0 ... -1.65e+03 -1.71e+03 -1.77e+03
    longitude  (y, x) float64 ...
    latitude   (y, x) float64 ...
Dimensions without coordinates: nbounds
Data variables:
    z_bnds     (z, nbounds) float64 ...
    thetao     (z, y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    so         (z, y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    tob        (y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    sob        (y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    ficeshelf  (y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    depfli     (y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    deptho     (y, x) float64 -1e+04 -1e+04 -1e+04 ... -1e+04 -1e+04 -1e+04
    crs        <U9 'epsg:3031'
Attributes:
    source:         eORCA025.L121-OPM021 NEMO simulation
    model time:     10 years average starting 01/01/2029
    grid:           ISMIP6
    contact:        P. Mathiot (IGE)
    creation date:  2021-07-08 12:14:28.025826

In [23]:
# print('load NEMO coordinate ...')
# ncid= nc.Dataset(cf_coord_NEMO,'r')
# rlatfNEMO = ncid.variables['gphif'][:,:].squeeze()      # debug
# rlonfNEMO = ncid.variables['glamf'][:,:].squeeze()      # debug
# ncid.close()
# print('')

# print('convert wgs84 to Antarctic Polar Stereographic ...')
# transformer_lltoxy=pyproj.Transformer.from_crs('epsg:4326','epsg:3031',always_xy=True)
# xfNEMO,yfNEMO = transformer_lltoxy.transform(rlonfNEMO,rlatfNEMO)     # debug
# print('')

# plt.pcolormesh(x2dISMIP6-4000., y2dISMIP6-4000., toto, vmin=-2.5,vmax=2,edgecolors='crimson')
# plt.pcolormesh(xfNEMO[0:370,0:-1],yfNEMO[0:370,0:-1],mask2dNEMO[1:370,1:-1]*np.nan,
#                     shading='flat',edgecolors='grey')

# plt.scatter(xNEMO,yNEMO,c=maskNEMO,marker='o')
# plt.scatter(x2dISMIP6, y2dISMIP6,c=mask,marker='s')

# plt.xlim([300000, 380000])
# plt.ylim([-1.32e6, -1.25e6])